In [1]:
import pickle
# Load later
with open('simulation_dataset_nbin.pkl', 'rb') as f:
    data = pickle.load(f)

In [ ]:
k=2

In [9]:
import numpy as np
import pandas as pd
data=pd.read_csv('simulated_dataset_r_positive.csv')

In [10]:
data.head()

,data,beta1,gamma1,alpha1,n_unique,p_0,p_k,p_p,y_mean,y_std,r_hat,p_nb,p0_nb,pk_nb,0_inflated,k_inflated
0,y x\n0 0 -0.470010\n1 1 1....,-2.0,-2.0,-2.0,5,0.50667,0.22000,0.27333,0.760000,0.904689,9.880000,0.480856,0.480856,0.131861,1.0,1.0
1,y x\n0 0 -1.021010\n1 0 0....,-2.0,-2.0,-2.0,4,0.52333,0.24667,0.23000,0.743333,0.864067,168.582438,0.476304,0.476304,0.131210,1.0,1.0
2,y x\n0 0 -1.526949\n1 0 -1....,-2.0,-2.0,-2.0,4,0.52667,0.25000,0.22333,0.733333,0.859078,114.853968,0.481426,0.481426,0.128926,1.0,1.0
3,y x\n0 0 -1.581881\n1 1 1....,-2.0,-2.0,-2.0,5,0.54000,0.17667,0.28333,0.666667,0.823295,39.866667,0.516255,0.516255,0.113764,1.0,1.0
4,y x\n0 2 -0.342659\n1 2 0....,-2.0,-2.0,-2.0,5,0.49667,0.23667,0.26666,0.780000,0.891206,42.702254,0.461644,0.461644,0.138611,1.0,1.0


In [22]:
for i in data.itertuples(index=False):
    lol=i
    df = i.data
    
    print(df['y'])
    break

TypeError: string indices must be integers

In [20]:
df

'     y         x\n0    0 -0.470010\n1    1  1.863750\n2    2 -0.177126\n3    2  0.247797\n4    0 -0.295470\n..  ..       ...\n295  1  0.766390\n296  2  0.643993\n297  0  0.925336\n298  0 -1.241522\n299  0 -2.139884\n\n[300 rows x 2 columns]'

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

df = df
    
# Add intercept column
df['intercept'] = 1

# Prepare features and target
features = df.loc[:, ['intercept', 'x']]     
target = df['y']
    
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
        features, target, test_size=0.3, random_state=42
)

TypeError: 'str' object does not support item assignment

In [ ]:
import statsmodels.api as sm

poisson_model = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
poisson_pred = poisson_model.predict(X_test)
poisson_mse = mean_squared_error(y_test, poisson_pred)
poisson_mae = mean_absolute_error(y_test, poisson_pred)
poisson_r2 = r2_score(y_test, poisson_pred)  
poisson_llf = poisson_model.llf  # Log-likelihood at maximum
poisson_aic = poisson_model.aic  # AIC = -2*LL + 2*df_model
poisson_bic = poisson_model.bic  # BIC = -2*LL + log(n)*df_model
poisson_mse, poisson_mae, poisson_r2, poisson_llf      

KeyboardInterrupt: 

In [8]:
nb2_model = sm.NegativeBinomial(y_train, X_train).fit()
nb_pred = nb2_model.predict(X_test)
nb_mse = mean_squared_error(y_test, nb_pred)
nb_mae = mean_absolute_error(y_test, nb_pred)
nb_r2 = r2_score(y_test, nb_pred)

Optimization terminated successfully.
         Current function value: 1.046987
         Iterations: 23
         Function evaluations: 24
         Gradient evaluations: 24


In [ ]:
from models.ZKIP import *

zkip_model = ZKIP_EM(k_inflated=k)
zkip_model.fit(X_train.values, y_train.values)
zkip_em_predictions = zkip_model.predict_expected(X_test.values)
zkip_em_mse = mean_squared_error(y_test, zkip_em_predictions)
zkip_em_mae = mean_absolute_error(y_test, zkip_em_predictions)
zkip_em_r2 = r2_score(y_test, zkip_em_predictions)

In [ ]:
from models.ZKINB import *

zkinb_model = ZkINB_EM()
zkinb_model.fit_em(y_train.values, X_train.values, X_train.values, k)
zkinb_em_predictions = zkinb_model.predict(X_test.values, X_test.values)
zkinb_em_mse = mean_squared_error(y_test, zkinb_em_predictions)
zkinb_em_mae = mean_absolute_error(y_test, zkinb_em_predictions)
zkinb_em_r2 = r2_score(y_test, zkinb_em_predictions)

In [ ]:
from models.ZIP import *

beta_params, gamma_params = ZIP_EM(y_train.values, X_train.values, X_train.values)
zip_em_predictions = predict_mean(X_test.values, X_test.values, beta_params, gamma_params)
zip_em_mse = mean_squared_error(y_test, zip_em_predictions)
zip_em_mae = mean_absolute_error(y_test, zip_em_predictions)
zip_em_r2 = r2_score(y_test, zip_em_predictions)

In [ ]:
from models.ZkICMP import *

zkicmp_model = ZkICMP(k=k)
zkicmp_model.fit(X_train.values, y_train.values)
pred_results = zkicmp_model.predict(X_test.values)
_, _, zkicmp_predictions, _ = pred_results
zkicmp_mse = mean_squared_error(y_test, zkicmp_predictions)
zkicmp_mae = mean_absolute_error(y_test, zkicmp_predictions)
zkicmp_r2 = r2_score(y_test, zkicmp_predictions)

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor
# HistGradientBoostingRegressor with Poisson loss
poisson_GB = HistGradientBoostingRegressor(
            loss='poisson',        # This is the key - uses Poisson likelihood
            random_state=42,
            max_iter=100,
            learning_rate=0.1,
            max_depth=6,
            min_samples_leaf=20    # Good for count data to prevent overfitting
        )
poisson_GB.fit(X_train.reshape(-1, 1), y_train)
y_pred_gb = poisson_GB.predict(X_test.reshape(-1, 1))
mse = mean_squared_error(y_test, y_pred_gb)
mae = mean_absolute_error(y_test, y_pred_gb)
r2 = r2_score(y_test, y_pred_gb)


In [ ]:
from sklearn.ensemble import RandomForestRegressor 
# RandomForestRegressor
rf = RandomForestRegressor(
        n_estimators=300,
        criterion="poisson",
        min_samples_leaf=5,
        n_jobs=-1,
        random_state=42
    )
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
mse = mean_squared_error(y_test, y_pred_rf)
mae = mean_absolute_error(y_test, y_pred_rf)
r2 = r2_score(y_test, y_pred_rf)
mse, mae, r2